In [1]:
import pandas as pd
import numpy as np

In [2]:
neg = pd.read_csv('negative_set.bed', delimiter='\t', header=None)

In [3]:
enh = pd.read_csv('GM12878.bed', delimiter='\t', header=None)

In [4]:
enh.columns = ['chr', 'cor1', 'cor2', 'score']

In [5]:
neg.columns = ['chr', 'cor1', 'cor2']

In [6]:
enh.drop('score', axis=1, inplace=True)

In [42]:
np.unique(neg.chr)

array(['chr10', 'chr11', 'chr12', 'chr13', 'chr15', 'chr16', 'chr17',
       'chr18', 'chr19', 'chr2', 'chr20', 'chr22', 'chr3', 'chr4', 'chr5',
       'chr6', 'chr7', 'chr8', 'chr9'], dtype=object)

In [40]:
np.unique(enh.chr)

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21',
       'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9'],
      dtype=object)

In [39]:
enh = enh[enh.chr != 'chrX']

In [41]:
neg = neg[neg.chr != 'chrX']
neg = neg[neg.chr != 'chrY']

In [52]:
from Bio import SeqIO
import gzip

df_fasta = pd.DataFrame(columns=['ID', 'sequence'])
with gzip.open("GRCh37.primary_assembly.genome.fa.gz", "rt") as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        sequence = record.seq
        df_fasta = pd.concat([df_fasta, pd.DataFrame.from_records([{'ID':identifier, 'sequence':sequence}])], ignore_index=True)

In [115]:
enhancers = []
for i in range(len(enh)):
    obs = enh.loc[i]
    seq = df_fasta[df_fasta.ID == obs['chr']].reset_index().sequence[0]
    enhancers.append(seq[obs['cor1']:obs['cor2']])

In [119]:
enh['Sequence'] = enhancers

In [123]:
enh['Enhancer'] = np.ones(len(enh))

In [132]:
enhancers = []
for i in range(len(neg)):
    obs = neg.loc[i]
    seq = df_fasta[df_fasta.ID == obs['chr']].reset_index().sequence[0]
    enhancers.append(seq[obs['cor1']:obs['cor2']])

In [133]:
len(enhancers)

78479

In [134]:
neg['Sequence'] = enhancers

In [135]:
neg['Enhancer'] = np.zeros(len(neg))

In [155]:
set = pd.concat([neg, enh]).reset_index()

In [170]:
set.reset_index()

,index,chr,cor1,cor2,Sequence,Enhancer
0,2,chr10,72312,74222,"(C, T, G, C, C, A, C, C, G, T, G, C, C, T, G, ...",0.0
1,3,chr10,84717,85177,"(A, G, A, A, G, T, T, G, A, A, T, C, T, C, T, ...",0.0
2,4,chr10,90499,91949,"(A, G, G, T, C, A, G, G, T, G, A, T, C, T, G, ...",0.0
3,5,chr10,104967,107977,"(T, T, T, T, C, C, C, C, A, C, T, T, G, G, T, ...",0.0
4,6,chr10,114029,114779,"(G, A, G, A, G, C, T, G, T, T, A, A, A, G, C, ...",0.0
...,...,...,...,...,...,...
120887,127034,chr9,140651150,140652330,"(T, T, G, G, G, G, T, A, G, A, G, A, T, T, T, ...",1.0
120888,127035,chr9,140702130,140703100,"(T, G, T, A, T, T, T, T, T, A, G, T, A, G, A, ...",1.0
120889,127036,chr9,140703310,140704120,"(G, T, T, G, C, A, G, T, G, A, A, C, C, A, A, ...",1.0
120890,127037,chr9,140710520,140711890,"(G, G, C, A, G, C, T, T, C, C, T, G, C, C, G, ...",1.0


In [158]:
set = set.drop('index', axis=1)

In [178]:
set.to_csv('enahncer_set.csv', index=False)

In [166]:
set.shape

(127039, 5)

In [ ]:
nukl = ['A', 'T', 'C', 'G']

In [167]:
for i in range(len(set)):
    if 'N' in set.Sequence[i]:
        set.drop(i, axis=0, inplace=True)

In [174]:
set = set.reset_index()

In [176]:
set.drop('index', axis=1, inplace=True)

In [177]:
set

,chr,cor1,cor2,Sequence,Enhancer
0,chr10,72312,74222,"(C, T, G, C, C, A, C, C, G, T, G, C, C, T, G, ...",0.0
1,chr10,84717,85177,"(A, G, A, A, G, T, T, G, A, A, T, C, T, C, T, ...",0.0
2,chr10,90499,91949,"(A, G, G, T, C, A, G, G, T, G, A, T, C, T, G, ...",0.0
3,chr10,104967,107977,"(T, T, T, T, C, C, C, C, A, C, T, T, G, G, T, ...",0.0
4,chr10,114029,114779,"(G, A, G, A, G, C, T, G, T, T, A, A, A, G, C, ...",0.0
...,...,...,...,...,...
120887,chr9,140651150,140652330,"(T, T, G, G, G, G, T, A, G, A, G, A, T, T, T, ...",1.0
120888,chr9,140702130,140703100,"(T, G, T, A, T, T, T, T, T, A, G, T, A, G, A, ...",1.0
120889,chr9,140703310,140704120,"(G, T, T, G, C, A, G, T, G, A, A, C, C, A, A, ...",1.0
120890,chr9,140710520,140711890,"(G, G, C, A, G, C, T, T, C, C, T, G, C, C, G, ...",1.0


In [164]:
'N' in set.Sequence[120]

False

In [ ]:
set.to_csv('enahncer_set.csv', index=False)